# Single Cell RNA Seq data

## Conlsusion
-  The expression level of CIT2 changed significently (p-value=0.13). Therefore, the analysis implies that the stress turns on the retrograde response.
- BMH has highes expression level among the RTG genes, which locates in $10^{3}$
- RTG1 has the lowese expression level. Our RTG model needs further verification becuase we assume the expression level of RTG1 is much larger than others.

## Reference
- Single-cell RNA sequencing reveals intrinsic and extrinsic regulatory heterogeneity in yeast responding to stress. [[link](https://journals.plos.org/plosbiology/article?id=10.1371/journal.pbio.2004050)]
- D3E : Discrete distributional differential expression (D3E) - a tool for gene expression analysis of single-cell RNA-seq data. [[link](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-016-0944-6)]

## Plotting Single Cell RNA Data

### Method
- Original data is separated into "Stressed" and "Unstressed"
- Applied D3E tool to Seq data

### Conclusion
- Dropout event inferences the correlation analysis
- Reqiure Single cell analysis tool
    - D3E 
    - [Summary of SC-Seq tool until 2018](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-019-2599-6/tables/1)

In [ ]:
import os
os.listdir("data/")

In [ ]:
# Access original xlsx file
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Alias of gene names
alias = {
    "RTG1": "YOL067C",
    "RTG2": "YGL252C",
    "RTG3": "YBL103C",
    "BMH1": "YER177W",
    "MKS1": "YNL076W",
    "CIT2": "YCR005C"
}

# Read original Single-cell seq data
data = "data/SingleCellRNAseq_yeast_GSE102475_LabelSep.txt"
df = pd.read_table(data)

dft = df.set_index("GeneID")
dft = dft.transpose()
dft = dft.filter(items= list(alias.values()))
dft.columns = list(alias.keys())

label = []
conds = ["Stressed", "Unstressed"] # Two conditions 
for i in dft.index.values:
    for cond in conds:
        if cond in i:
            label.append(cond)
dft["condition"] = label


In [ ]:
sns.pairplot(dft, hue="condition", kind="scatter" )

In [ ]:
sns_plot = sns.pairplot(dft, hue="condition", kind="reg")
sns_plot.savefig("img/rna_pairPlot.pdf")

In [ ]:
# Show dropout events
dft.drop( ["condition"], axis=1)

# D3E: Single-cell seq data analysis

<img src="https://media.springernature.com/lw685/springer-static/image/art%3A10.1186%2Fs12859-016-0944-6/MediaObjects/12859_2016_944_Fig1_HTML.gif?as=webp">

*Transcriptional bursting model*

Sequencing data is analysed by D3E tool ([github](https://github.com/hemberg-lab/D3E))

[The results](data/SingelCellRNAseq_yeast_GSE102475_LabelSep.out) are derived by the following bash command:

```sh
python D3ECmd.py SingelCellRNAseq_yeast_GSE102475_LabelSep.txt  SingelCellRNAseq_yeast_GSE102475_LabelSep.out  Stressed Unstressed -m 1 -t 0 -z 0 -n 1 -v 
```

In [ ]:
# Read analysis
import pandas as pd
d3e = pd.read_table("data/SingleCellRNAseq_yeast_GSE102475_LabelSep.out")
d3e = d3e.set_index("#GeneID")
d3eT = d3e.transpose()
# Display
f = d3eT.filter( list(alias.values()), axis=1)
f.columns = list(alias.keys())
f

In [ ]:
# Save results
d3e.to_csv("data/d3e_SC_seq_resp.csv")
f.to_csv("data/d3e_SC_resp_RtgGenes.csv")

### Mean and Std are not simply the calculation of read-count table

In [ ]:
"""
Show that the mean value and std of d3e result are not simply the mean and std of read-count table
"""
import numpy as np

# Set a gene
GeneName = "RTG3"

# Retrieve data
s = df[df["GeneID"] == alias[GeneName]].transpose()
d = s.to_dict()
key = list(d.keys())[0]
val = list(d[key].values())
del val[0]
val = np.array(val)


# Cal mean and cv
mean = np.mean(val[val!=0.0])
cv = np.std(val[val!=0.0])

# Print
print("TABLE CALCULATION")
print("MEAN: {}(Table)/\t {}(mu1)/\t {}(mu2)".format(mean, f[GeneName]["mu1"], f[GeneName]["mu2"]))
print("STD: {}(Table)/\t {}(cv1)/\t {}(cv2)".format(cv,f[GeneName]["cv1"],f[GeneName]["cv2"]))

# Plotting D3E results

In [ ]:
import pandas as pd
d3e = pd.read_csv("data/d3e_SC_resp_RtgGenes.csv", index_col=0)
d3e

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# DATA
st_means, st_std = d3e[d3e.index == "mu1"].values, d3e[d3e.index == "cv1"].values
ust_means, ust_std = d3e[d3e.index == "mu2"].values, d3e[d3e.index == "cv2"].values

pvalues = d3e[d3e.index == "p-value"].values
name_of_genes = list(d3e.keys())

# SETTING
st_means, st_std, ust_means, ust_std, pvalues = st_means.flatten(), st_std.flatten(), ust_means.flatten(), ust_std.flatten(), pvalues.flatten()

ind = np.arange(len(st_means))  # the x locations for the groups
width = 0.35  # the width of the bars

## Add p value
name_p = [ name + "\n(p-value:{0:.2f})".format(p) for name, p in zip(name_of_genes, pvalues )]
name_of_genes = name_p

# PLOTTING
fig, ax = plt.subplots(figsize=(10,7))
rects1 = ax.bar(ind - width/2, st_means, width, yerr=st_std,
                label='Stressed')
rects2 = ax.bar(ind + width/2, ust_means, width, yerr=ust_std,
                label='Unstressed')

## Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Expression Level (normalized)')
ax.set_title('Stressed response of RTG gnes')
ax.set_xticks(ind)
ax.set_xticklabels( name_of_genes)
ax.legend()


def autolabel(rects, xpos='center'):
    """
    Attach a text label above each bar in *rects*, displaying its height.

    *xpos* indicates which side to place the text w.r.t. the center of
    the bar. It can be one of the following {'center', 'right', 'left'}.
    """

    ha = {'center': 'center', 'right': 'left', 'left': 'right'}
    offset = {'center': 0, 'right': 1, 'left': -1}

    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(offset[xpos]*3, 3),  # use 3 points offset
                    textcoords="offset points",  # in both directions
                    ha=ha[xpos], va='bottom', color="black")


autolabel(rects1, "left")
autolabel(rects2, "right")





fig.tight_layout()
plt.yscale("log")
plt.show()

fig.savefig("img/stressed_response_mean.pdf")
fig.savefig("img/stressed_response_mean.png", dpi=100)